In [1]:
// I'm keeping this disabled for now, since I don't really need it is buggy
//%jsroot on

In [2]:
TFile* results = new TFile("../histograms/kstar-decay-hist.root","READ");


TH1F* ParticleDist = (TH1F*)results->Get("ParticleDist");
TH1F* MomentumDist =(TH1F*)results->Get("MomentumDist");

c = new TCanvas()

(TCanvas *) @0x7fb19cf8e8a0


In [3]:

MomentumDist->Draw();

c->Draw()
    

In [4]:
ParticleDist->Draw();
c->Draw()